In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qbayes_tools import generate_cond_keys

In [9]:
lesserdata = pd.read_csv('data/lesser_model_data.csv')

In [10]:
"""
IMPORTANT NOTE Y'ALL::maybe once we have all three models we can move the get_lesser_model_states
to some sort of get states code and just make this a place to get general probabilities 
(and call get_states and probabilities functions from a network specific circuit builder directly,
OR a general circuit builder would be the dream!!)
"""

"\nIMPORTANT NOTE Y'ALL::maybe once we have all three models we can move the get_lesser_model_states\nto some sort of get states code and just make this a place to get general probabilities \n(and call get_states and probabilities functions from a network specific circuit builder directly,\nOR a general circuit builder would be the dream!!)\n"

In [11]:
def get_lesser_model_states():
    statedataStayHome = {'MarHome' : lesserdata['MarHome'], 'AprHome' : lesserdata['AprHome'], 'MayHome' : lesserdata['MayHome'], 'JunHome' : lesserdata['JunHome']}
    statesStayHome = pd.DataFrame(data=statedataStayHome)

    statedataTests = {'MarTest' : lesserdata['MarTest'], 'AprTest' : lesserdata['AprTest'], 'MayTest' : lesserdata['MayTest'], 'JunTest' : lesserdata['JunTest']}
    statesTests = pd.DataFrame(data=statedataTests)

    statedataCases = {'MarCases' : lesserdata['MarCases'], 'AprCases' : lesserdata['AprCases'], 'MayCases' : lesserdata['MayCases'], 'JunCases' : lesserdata['JunCases']}
    statesCases = pd.DataFrame(data=statedataCases) # 0 = increasing. 1 = flat or decreasing
    
    return np.ndarray.flatten(np.array(statesCases)), np.ndarray.flatten(np.array(statesTests)), np.ndarray.flatten(np.array(statesStayHome))

In [21]:
mallarddata = pd.read_csv('data/mallardmodeldata.csv')
def get_mallard_model_states():
    Cases = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarCases':mallarddata['MarCases'], 'AprCases':mallarddata['AprCases'], 'MayCases':mallarddata['MayCases'], 'JunCases':mallarddata['JunCases']})))
    Test = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarTest':mallarddata['MarTest'],'AprTest':mallarddata['AprTest'],'MayTest':mallarddata['MayTest'], 'JuneTest':mallarddata['JunTest']})))
    Mask = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarMask':mallarddata['MarMask'],'AprMask':mallarddata['AprMask'],'MayMask':mallarddata['MayMask'],'JunMask':mallarddata['JunMask']})))
    Work = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarWork':mallarddata['MarWork'],'AprWork':mallarddata['AprWork'],'MayWork':mallarddata['MayWork'],'JunWork':mallarddata['JunWork']})))
    Rec = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarRec':mallarddata['MarRec'],'AprRec':mallarddata['AprRec'],'MayRec':mallarddata['MayRec'],'JunRec':mallarddata['JunRec']})))
    Death = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarDeath':mallarddata['MarDeath'],'AprDeath':mallarddata['AprDeath'],'MayDeath':mallarddata['MayDeath'],'JunDeath':mallarddata['JunDeath']})))
    Age = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarAge':mallarddata['MarAge'],'AprAge':mallarddata['AprAge'],'MayAge':mallarddata['MayAge'],'JunAge':mallarddata['JunAge']})))
    return Cases,Test,Mask,Work,Rec,Death,Age


In [23]:
alabiodata = pd.read_csv('data/alabiomodeldata.csv')
def get_alabio_model_states():
    Cases = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarCases':alabiodata['MarCases'], 'AprCases':alabiodata['AprCases'], 'MayCases':alabiodata['MayCases'], 'JunCases':alabiodata['JunCases']})))
    Test = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarTest':alabiodata['MarTest'],'AprTest':alabiodata['AprTest'],'MayTest':alabiodata['MayTest'], 'JuneTest':alabiodata['JunTest']})))
    Mask = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarMask':alabiodata['MarMask'],'AprMask':alabiodata['AprMask'],'MayMask':alabiodata['MayMask'],'JunMask':alabiodata['JunMask']})))
    Work = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarWork':alabiodata['MarWork'],'AprWork':alabiodata['AprWork'],'MayWork':alabiodata['MayWork'],'JunWork':alabiodata['JunWork']})))
    Rec = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarRec':alabiodata['MarRec'],'AprRec':alabiodata['AprRec'],'MayRec':alabiodata['MayRec'],'JunRec':alabiodata['JunRec']})))
    Death = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarDeath':alabiodata['MarDeath'],'AprDeath':alabiodata['AprDeath'],'MayDeath':alabiodata['MayDeath'],'JunDeath':mallarddata['JunDeath']})))
    Age = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarAge':alabiodata['MarAge'],'AprAge':alabiodata['AprAge'],'MayAge':alabiodata['MayAge'],'JunAge':alabiodata['JunAge']})))
    Race = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarRace':alabiodata['MarRace'],'AprRace':alabiodata['AprRace'],'MayRace':alabiodata['MayRace'],'JunRace':alabiodata['JunRace']})))
    Poverty = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarPoverty':alabiodata['MarPoverty'],'AprPoverty':alabiodata['AprPoverty'],'MayPoverty':alabiodata['MayPoverty'],'JunPoverty':alabiodata['JunPoverty']})))
    Health = np.ndarray.flatten(np.array(pd.DataFrame(data={'MarHealth':alabiodata['MarHealth'],'AprHealth':alabiodata['AprHealth'],'MayHealth':alabiodata['MayHealth'],'JunHealth':alabiodata['JunHealth']})))
    return Cases, Test, Mask, Work, Rec, Death, Age, Race, Poverty, Health
    

In [5]:
def get_probabilities(state_data):
    ############################################
    # USE THIS FUNCTION TO FIND THE PROBABILITIES FOR AN INDIVIDUAL NODE IN THE NETWORK
    
    ### INPUT ###
    # state_data: 1d array indicating the state corresponding to individual data points
    #             (note: for an n-state system the array can contain the integers 
    #             0, 1, ..., n-1, but must contain n-1 at least once) [note to contributors: 
    #             maybe add an optional input "n" to specify a maximum state that is never attained?]

    ### OUTPUT ###
    # probs: array of length n with probabilities for states 0 through n-1 in increasing order
    ############################################

    num_total = len(state_data)  #total number of data points with which to calculate probabilities
    s = max(state_data) + 1  #number of different states will be the maximum integer included in the state data plus one

    probs = np.zeros(s)
    for i in range(s): #loop through different allowed states
        probs[i] = np.shape(np.where(state_data == i))[1]/num_total

    assert round(np.sum(probs), 3) == 1. 
    
    return probs

In [18]:
def get_conditional_probability(child, *ps):
    ############################################
    ### THIS FUNCTION CALCULATES CONDITIONAL PROBABILITIES FOR CHILD NODE
    ### THAT HAS s_m STATES AND m PARENT NODES EACH WITH s_i STATES WHERE i = 0, ..., m-1
    
    ### INPUTS ###
    # child    array    1d array with n entries specifying the state of the child node at each data point
    # *ps      array(s) variable number of 1d arrays with n entries each, specifying the states of parent nodes at each data point
    
    ### OUTPUT ###
    # a dictionary of conditional probabilities
    ############################################

    #we might want to add some assert statements checking that all inputs have the same shape
    
    m = len(ps) #number of parent nodes
    s_0 = max(child) + 1 #number of child nodes

    s_i = np.zeros(m)
    for i in range(m):
        s_i[i] = max(ps[i]) + 1 #number of states for this node will be the maximum state noted in that data plus one

    keys = generate_cond_keys(s_0,s_i.astype(int))

    cond_probs = {} #initialize a dictionary for conditional probabilities
    for key in keys:
        numer, tot = 0, 0
        n = len(child)
        for i in range(n):
            all_ps = True
            for j in range(len(ps)):
                p = ps[j]
                if p[i] != int(key.split("|")[1].split(",")[j]):
                    all_ps = False
                    break
            if all_ps:
                tot += 1
                if child[i] == int(key.split("|")[0]):
                    numer += 1
        cond_probs.update({key : numer/tot})
  
    return cond_probs

In [19]:
C,B,A = get_lesser_model_states()

print(get_probabilities(C))
print(get_conditional_probability(C, B, A))

[0.755 0.245]
{'0|0,0': 0.8181818181818182, '0|0,1': 0.7068965517241379, '0|1,0': 0.8412698412698413, '0|1,1': 0.23076923076923078, '1|0,0': 0.18181818181818182, '1|0,1': 0.29310344827586204, '1|1,0': 0.15873015873015872, '1|1,1': 0.7692307692307693}


In [ ]:
# I HAVENT UPDATED MAKING GRAPH FOR CIRCUIT INPUT TO BE GENERAL BUT ALSO IDK IF THIS IS STILL THE WAY WE WANT TO DO THIS...??    
"""
    # Make graph for circuit input
    ProbA1, ProbA0 = get_probabilities(Astates)
    ProbB1, ProbB0 = get_probabilities(Bstates)
    
    graph = { 'StayAtHome': ([], [ProbA0, ProbA1]), # P(A = 0), P(A = 1)
                'Testing': ([], [ProbB0, ProbB1]),  #P(B = 0), P(B = 1)
                #P(C=0|A=0,B=0), P(C=1|A=0,B=0), P(C=0|A=0,B=1), P(C=1|A=0,B=1), P(C=0|A=1,B=0), P(C=1|A=1,B=0), P(C=0|A=1,B=1), P(C=1|A=1,B=1)
                'Cases': (['StayAtHome','Testing'], [P000, P100, P010, P110, P001, P101, P011, P111])
              }
    
    return states, conditional_probabilities, graph
    """

In [6]:
# example: results from running simple model on simulator: 
# {'000': 2783, '001': 1240, '100': 603, '111': 815, '110': 294, '010': 1712, '101': 485, '011': 260}

def get_marginal_0probabilities(state_counts):
    #state_counts: dict, counts for each state from network result (should have 2^n entries)
    #marg_probs: array of length n, marginal probabilities that each qubit is 0,
        #from most significant to least significant qubit
    
    n = len(list(state_counts.keys())[0]) #number of qubits
    prob = np.zeros(n)
    total = sum(state_counts.values())

    for i in range(n):
        for key in state_counts:
            if int(key[i]) == 0:
                prob[i] += state_counts[key]
        prob[i] = prob[i]/total
    
    return prob




These calculations agree with marginal_probabilities function, yay !

PC0_calc = (2783+1240+1712+260)/(2783+1240+603+815+294+1712+485+260)

print(PC0_calc)

PB0_calc = (2783+1240+485+603)/(2783+1240+603+815+294+1712+485+260)

print(PB0_calc)

PA0_calc = (2783+603+294+1712)/(2783+1240+603+815+294+1712+485+260)

print(PA0_calc)


In [1]:
# get marginal probabilities for simulation results - lesser model

results = {'000': 2783, '001': 1240, '100': 603, '111': 815, '110': 294, '010': 1712, '101': 485, '011': 260}
#margProb = marginal_probabilities(results)
#margCompare = marginal_probabilities_general(results)
#print(margProb)
#print(margCompare)

In [6]:
def func(**counts):
    return counts["c001"]

In [7]:
func(c000=4928, c001=82743, c100=48937, c111=9288842)

82743